In [ ]:
##############################################################################
# PART 1: Imports
##############################################################################
from experiments.base_heb_cpu import BaseHebCPU
from models.hebbian_network import HebbianNetwork
from utils.experiment_parser import *
from utils.experiment_comparer import *
from utils.experiment_logger import *
from utils.experiment_timer import *
from utils.experiment_stats import *


In [ ]:
##############################################################################
# PART 2: Create logs for experiment
##############################################################################



In [ ]:
##############################################################################
# PART 3: Parse arguments for the experiment
##############################################################################

# Simulate the command line arguments
args_dict = {
    '--is_training': True,
    '--data_name': 'MNIST',
    '--train_data': 'data/mnist/train-images.idx3-ubyte',
    '--train_label': 'data/mnist/train-labels.idx1-ubyte',
    '--test_data': 'data/mnist/t10k-images.idx3-ubyte',
    '--test_label': 'data/mnist/t10k-labels.idx1-ubyte',
    '--train_filename': 'data/mnist/mnist_train.csv',
    '--test_filename': 'data/mnist/mnist_test.csv',
    '--input_dim': 784,
    '--heb_dim': 64,
    '--output_dim': 10,
    '--heb_lamb': 1,
    '--heb_gam': 0.99,
    '--cla_lamb': 1,
    '--lr': 0.005,
    '--eps': 0.01,
    '--batch_size': 1,
    '--epochs': 1,
    '--test_epochs': 1,
    '--device_id': 'cpu',
    '--local_machine': True
}

# Convert the dictionary to a list of arguments
args_list = []
for k, v in args_dict.items():
    args_list.append(k)
    args_list.append(str(v))
    
ARGS = parse_arguments(args_list)

In [ ]:
##############################################################################
# PART 4: Create and run experiment
##############################################################################
# Get arguments
#ARGS = parse_arguments()

# Experiments setup
lambda_test = [1]
    
lambda_test.sort()
print(lambda_test)

for l in lambda_test:
    ARGS.heb_lamb = l
    
    model = HebbianNetwork(ARGS)
    experiment = BaseHebCPU(model, ARGS, f'cpu-{l}')
    test_acc, train_acc = experiment.run()
    experiment.cleanup()
    
    print(f"Epoch: {ARGS.epochs} || Lambda: {l} || Test Acc: {test_acc} || Train Acc: {train_acc}")

In [3]:
# Simulate the command line arguments
args_dict = {
    '--color': 'RED'
}

# Convert the dictionary to a list of arguments
args_list = []
for k, v in args_dict.items():
    args_list.append(k)
    args_list.append(str(v))

In [4]:
import argparse
from enum import Enum

class Color(Enum):
    RED = 'RED'
    BLUE = 'BLUE'
    YELLOW = 'YELLOW'

def main():
    # Create the parser
    parser = argparse.ArgumentParser(description="Process some colors.")
    
    # Add an argument for color
    parser.add_argument('--color', type=str, help='Color to be used (RED, BLUE, YELLOW)', default='RED')
    
    # Parse the arguments
    args = parser.parse_args(args_list)
    
    # Convert the string argument to uppercase and then to the corresponding Color enum member
    try:
        selected_color = Color[args.color.upper()]
        print(f"Selected color: {selected_color}")
    except KeyError:
        print(f"Invalid color: {args.color}. Valid options are: RED, BLUE, YELLOW.")

if __name__ == "__main__":
    main()


Selected color: Color.RED
